# Tuberculosis Detection

In [1]:
from utils import load_dataframe
import fastai
from fastai.vision.all import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score

ModuleNotFoundError: No module named 'fastai'

## Generate the dataframe

In [ ]:
dataset_description_path = "dataset_description.yaml"

In [ ]:
#Generate dataset from description
tb_df = load_dataframe(dataset_description_path)

In [ ]:
tb_df

In [ ]:
train_test_df, val_df = train_test_split(tb_df, test_size=0.1)

In [ ]:
print("Train/Test Dataframe:", train_test_df.shape)
print("Validation Dataframe:", val_df.shape)

## Configuring the Dataloader

In [ ]:
def train_test_splitter(dataframe):
    train_df, test_df = train_test_split(dataframe, test_size=0.2)
    train_df.index.tolist()
    test_df.index.tolist()
    return (train_df, test_df)
def get_image_input(row):
    return row['path']
def get_labels(row):
    return row['abnormal'],row['tuberculosis']
def get_abnormal(row):
    return row['abnormal']
def get_tuberculosis(row):
    return row['tuberculosis']

In [ ]:
# tfms = [Normalize.from_stats([0.0692], [0.2051]),  # Normalize the images with the specified mean and standard deviation
#         *aug_transforms(do_flip=False, size=128)]  # Add default transformations except for horizontal flip    
# dblock = DataBlock(
#   blocks=(ImageBlock(cls=PILImageBW), *(2*[CategoryBlock])),      # one image input and three categorical targets
#   getters=[ColReader('path'),   # image input
#            ColReader('abnormal'),                            # label 1
#            ColReader('tuberculosis')],                       # label 2
#   splitter=train_test_splitter,                 # train/validation split
#   batch_tfms=tfms)                                           # Add normalization and augmentations  
# dls = dblock.dataloaders(train_test_df, bs=64)   # Create the dataloaders
# dls.n_inp = 1  # Set the number of inputs

In [ ]:
tfms = [Normalize.from_stats([0.0692], [0.2051]),  # Normalize the images with the specified mean and standard deviation
        *aug_transforms(do_flip=False, size=128)]  # Add default transformations except for horizontal flip    
dblock = DataBlock(
  blocks=(ImageBlock(cls=PILImageBW), *(2*[CategoryBlock])),      # one image input and three categorical targets
  getters=[ColReader(1),   # image input
           ColReader(2),                            # label 1
           ColReader(3)],                       # label 2
  splitter=train_test_splitter,                 # train/validation split
  batch_tfms=tfms)                                           # Add normalization and augmentations  
dls = dblock.dataloaders(train_test_df, bs=5)   # Create the dataloaders
dls.n_inp = 1  # Set the number of inputs

In [ ]:
dls.show_batch(nrows=1, ncols=3)

In [ ]:
learn = cnn_learner(dls, resnet50, metrics=partial(accuracy_multi, thresh=0.5))